# Basic Sequence Models → RNNs (**Docs+Comments Edition**)


## 0) 준비
PyTorch가 없다면 자동 설치합니다.

In [78]:
try:
    import torch
except ImportError:
    !pip -q install torch --index-url https://download.pytorch.org/whl/cpu
import torch, random, re
from collections import defaultdict
print('PyTorch version:', torch.__version__)

PyTorch version: 2.4.1


## 1) 텍스트 n-gram 모델 (bi/tri/4-gram)
작은 문장 코퍼스로 n-gram 카운트를 만들고 다음 단어를 샘플링합니다.

핵심 포인트:
- `build_ngram_counts`: n-gram과 context 카운트 계산
- `next_probs`: 라플라스 스무딩 포함 다음 단어 확률 분포
- `generate_sentence`: `<s>`에서 출발해 `</s>`가 나오면 종료


In [79]:
# 작은 예시 코퍼스 정의
raw_sentences = [
    'the students opened their books',
    'the students opened their minds',
    'the teacher entered the classroom',
    'students love to solve problems',
    'the classroom is full of books',
    'open your mind and read more books',
    'solve more problems to learn more',
]

print('build_corpus 입력 문장 목록:')
for i, text in enumerate(raw_sentences, 1):
    print(f'  {i}: {text}')
print('build_corpus 입력 문장 수:', len(raw_sentences))


build_corpus 입력 문장 목록:
  1: the students opened their books
  2: the students opened their minds
  3: the teacher entered the classroom
  4: students love to solve problems
  5: the classroom is full of books
  6: open your mind and read more books
  7: solve more problems to learn more
build_corpus 입력 문장 수: 7


In [80]:
# 1. tokenize 함수와 예시
def tokenize(s: str):
    """간단 토크나이저: 소문자화 후, 알파벳/숫자/공백만 남기고 분할."""
    s = s.lower()
    s = re.sub(r'[^a-z0-9\s]', '', s)
    return s.split()

example_sentence = 'Open your Mind, and Read more books!!'
print('tokenize 입력 문장:', example_sentence)
token_output = tokenize(example_sentence)
print('tokenize 출력 토큰:', token_output)


tokenize 입력 문장: Open your Mind, and Read more books!!
tokenize 출력 토큰: ['open', 'your', 'mind', 'and', 'read', 'more', 'books']


In [81]:
# 2. build_corpus 함수와 예시
START, END = '<s>', '</s>'

def build_corpus(sentences):
    """문장 리스트를 `<s>`, `</s>` 포함 단일 토큰 리스트로 변환."""
    tokens = []
    for s in sentences:
        toks = tokenize(s)
        tokens.extend([START] + toks + [END])
    return tokens

corpus = build_corpus(raw_sentences)
print('build_corpus 입력 문장 수:', len(raw_sentences))
print('build_corpus 출력 토큰 시퀀스 앞 15개:', corpus[:15])


build_corpus 입력 문장 수: 7
build_corpus 출력 토큰 시퀀스 앞 15개: ['<s>', 'the', 'students', 'opened', 'their', 'books', '</s>', '<s>', 'the', 'students', 'opened', 'their', 'minds', '</s>', '<s>']


In [82]:
# 3. build_ngram_counts 함수와 예시
def build_ngram_counts(tokens, n: int = 3):
    """n-gram과 context 빈도수를 계산."""
    counts = defaultdict(int)
    ctx_counts = defaultdict(int)
    for i in range(len(tokens) - n + 1):
        ngram = tuple(tokens[i:i+n])
        ctx = ngram[:-1]
        counts[ngram] += 1
        ctx_counts[ctx] += 1
    return counts, ctx_counts

tri_counts, tri_ctx_counts = build_ngram_counts(corpus, n=3)
sample_ctx = ('<s>', 'the')
context_counts = {k: v for k, v in tri_counts.items() if k[:-1] == sample_ctx}
print('build_ngram_counts 입력 n:', 3)
print('build_ngram_counts 입력 토큰 수:', len(corpus))
print('build_ngram_counts 입력 컨텍스트:', sample_ctx)
print('build_ngram_counts 출력 카운트:', context_counts)


build_ngram_counts 입력 n: 3
build_ngram_counts 입력 토큰 수: 53
build_ngram_counts 입력 컨텍스트: ('<s>', 'the')
build_ngram_counts 출력 카운트: {('<s>', 'the', 'students'): 2, ('<s>', 'the', 'teacher'): 1, ('<s>', 'the', 'classroom'): 1}


In [83]:
# 4. next_probs 함수와 예시
def next_probs(context, counts, ctx_counts, vocab, delta: float = 1.0):
    """라플라스 스무딩으로 `P(next|context)` 계산."""
    ctx = tuple(context)
    total = ctx_counts.get(ctx, 0)
    V = len(vocab)
    probs = {}
    for w in vocab:
        c = counts.get(ctx + (w,), 0)
        probs[w] = (c + delta) / (total + delta * V) if (total + delta * V) > 0 else 1.0 / V
    s = sum(probs.values())
    for k in probs:
        probs[k] /= s
    return probs

vocab = sorted(set(corpus))
probs_example = next_probs(sample_ctx, tri_counts, tri_ctx_counts, vocab, delta=1.0)
top5 = sorted(probs_example.items(), key=lambda x: x[1], reverse=True)[:5]
print('next_probs 입력 컨텍스트:', sample_ctx)
print('next_probs 입력 delta:', 1.0)
print('next_probs 출력 확률 상위 5개:', top5)


next_probs 입력 컨텍스트: ('<s>', 'the')
next_probs 입력 delta: 1.0
next_probs 출력 확률 상위 5개: [('students', 0.10344827586206899), ('classroom', 0.06896551724137934), ('teacher', 0.06896551724137934), ('</s>', 0.03448275862068967), ('<s>', 0.03448275862068967)]


In [84]:
# 5. sample_dict 함수와 예시
def sample_dict(probs: dict):
    """확률 분포에서 1개 토큰 샘플링."""
    items, weights = list(probs.keys()), list(probs.values())
    return random.choices(items, weights=weights, k=1)[0]

toy_probs = {'A': 0.1, 'B': 0.2, 'C': 0.7}
print('sample_dict 입력 확률 분포:', toy_probs)
samples = [sample_dict(toy_probs) for _ in range(5)]
print('sample_dict 출력 샘플 5회:', samples)


sample_dict 입력 확률 분포: {'A': 0.1, 'B': 0.2, 'C': 0.7}
sample_dict 출력 샘플 5회: ['C', 'C', 'B', 'C', 'C']


In [85]:
# 6. generate_sentence 함수와 예시
def generate_sentence(n: int = 3, delta: float = 1.0, max_len: int = 20, seed=None):
    """n-gram LM으로 문장 생성. `<s>`*(n-1)에서 시작, `</s>`에서 종료."""
    counts, ctx_counts = build_ngram_counts(corpus, n)
    vocab = sorted(set(corpus))

    # 초기 컨텍스트 설정
    if seed is None:
        ctx = [START] * (n - 1)
    else:
        ctx = seed

    out = []
    for _ in range(max_len):
        probs = next_probs(ctx, counts, ctx_counts, vocab, delta)
        w = sample_dict(probs)
        if w == END:
            break
        if w not in (START, END):
            out.append(w)
        ctx = (ctx + [w])[1:]
    return ' '.join(out)


# ===== 예시 실행 =====
print("generate_sentence 목적: n-gram LM이 시작 컨텍스트에서 다음 단어를 반복 샘플링하는 과정을 보여줍니다.")

first_sentence_tokens = tokenize(raw_sentences[0])

for n in [2, 3, 4]:
    # 1) 기본 초기 컨텍스트
    default_seed = [START] * (n - 1)

    # 2) 첫 문장 기반 초기 컨텍스트
    example_seed = ([START] + first_sentence_tokens[:max(0, n - 2)])[:n - 1]

    print(f"generate_sentence 입력 값: n={n}, delta=1.0, max_len=15")
    print("  기본 초기 컨텍스트:", default_seed, "(n-1개의 `<s>` 토큰)")
    print("  첫 문장 기반 초기 컨텍스트:", example_seed)

    sentence_default = generate_sentence(n=n, delta=1.0, max_len=15, seed=default_seed)
    sentence_seeded = generate_sentence(n=n, delta=1.0, max_len=15, seed=example_seed)

    print("  출력 샘플 (기본 컨텍스트):", sentence_default)
    print("  출력 샘플 (첫 문장 컨텍스트):", sentence_seeded)

generate_sentence 목적: n-gram LM이 시작 컨텍스트에서 다음 단어를 반복 샘플링하는 과정을 보여줍니다.
generate_sentence 입력 값: n=2, delta=1.0, max_len=15
  기본 초기 컨텍스트: ['<s>'] (n-1개의 `<s>` 토큰)
  첫 문장 기반 초기 컨텍스트: ['<s>']
  출력 샘플 (기본 컨텍스트): books
  출력 샘플 (첫 문장 컨텍스트): love minds teacher classroom
generate_sentence 입력 값: n=3, delta=1.0, max_len=15
  기본 초기 컨텍스트: ['<s>', '<s>'] (n-1개의 `<s>` 토큰)
  첫 문장 기반 초기 컨텍스트: ['<s>', 'the']
  출력 샘플 (기본 컨텍스트): full solve teacher minds minds
  출력 샘플 (첫 문장 컨텍스트): classroom their to teacher read of the the their more full students read
generate_sentence 입력 값: n=4, delta=1.0, max_len=15
  기본 초기 컨텍스트: ['<s>', '<s>', '<s>'] (n-1개의 `<s>` 토큰)
  첫 문장 기반 초기 컨텍스트: ['<s>', 'the', 'students']
  출력 샘플 (기본 컨텍스트): minds books and books books full their opened more their entered minds students solve
  출력 샘플 (첫 문장 컨텍스트): to open learn their open teacher mind


## 2) 간단 RNN 문자 모델
임베딩 -> RNN -> Linear 구조로 다음 문자를 예측합니다.

- 입력: 선택한 문자 말뭉치를 정수 인덱스로 변환한 시퀀스(`data`).
- 학습 목표: t 시점 문자를 입력하면 t+1 문자를 맞추도록 RNN을 학습시킵니다.
- 출력: 학습된 RNN이 각 시점의 다음 문자 확률과 샘플링 텍스트를 제공합니다.

아래 셀에서 데이터 소스를 결정한 뒤 CharRNN을 학습하고, 임의의 시작 문자와 길이로 문자를 생성해 봅니다.


In [86]:
# (옵션) 외부 파일에서 문자 코퍼스 불러오기
from pathlib import Path
corpus_path = Path('data/simple_char_corpus.txt')
file_text = None
if corpus_path.exists():
    file_text = corpus_path.read_text()
    preview = file_text[:120].replace('\n', ' / ')
    print('문자 데이터 입력 경로:', corpus_path)
    print('문자 데이터 총 길이:', len(file_text))
    print('문자 데이터 앞 120자:', preview)
else:
    print('data/simple_char_corpus.txt가 없어서 기본 샘플을 사용합니다.')


문자 데이터 입력 경로: data/simple_char_corpus.txt
문자 데이터 총 길이: 339
문자 데이터 앞 120자: The RNN workshop explores basic sequence models. / We start from n-gram language models and gradually move to recurrent id


In [87]:
# 문자 데이터 준비
if file_text:
    text = file_text
    print('데이터 소스: 외부 파일')
    print('입력 문자 길이:', len(text))
else:
    text_samples = [
        'hello world hello pytorch',
        'pytorch makes building rnns easier',
        'sequence models learn next characters',
        'hello sequence modeling with pytorch',
        'rnn and gru handle sequential data',
    ]
    print('데이터 소스: 기본 샘플 리스트')
    for i, sentence in enumerate(text_samples, 1):
        print(f'  {i}: {sentence}')
    text = '\n'.join(text_samples) + '\n'
    print('결합된 입력 문자 길이:', len(text))

chars = sorted(set(text))
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}
data = torch.tensor([stoi[c] for c in text], dtype=torch.long)
print('고유 문자 수 (출력 차원):', len(chars))
print('정수 시퀀스 길이:', data.numel())


데이터 소스: 외부 파일
입력 문자 길이: 339
고유 문자 수 (출력 차원): 35
정수 시퀀스 길이: 339


In [88]:
# 1. CharRNN 클래스와 예시
import torch.nn as nn

class CharRNN(nn.Module):
    """문자 단위 RNN 언어 모델."""
    def __init__(self, vocab_size: int, hidden_size: int = 32):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h=None):
        """(B, T) 입력을 (B, T, V) 로짓으로 변환."""
        x = self.embed(x)
        out, h = self.rnn(x, h)
        return self.fc(out), h

char_model = CharRNN(len(chars))
sample_input = data[:10].unsqueeze(0)  # (1, 10)
print('CharRNN forward 입력 텐서 shape:', sample_input.shape)
logits, hidden = char_model(sample_input)
print('CharRNN forward 출력 로짓 shape:', logits.shape)


CharRNN forward 입력 텐서 shape: torch.Size([1, 10])
CharRNN forward 출력 로짓 shape: torch.Size([1, 10, 35])


In [89]:
# 2. train_char_rnn 함수와 예시
def train_char_rnn(model, data, seq_len=16, epochs=100, lr=1e-2):
    """한 글자 시프트된 타깃으로 교차엔트로피 학습."""
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    crit = nn.CrossEntropyLoss()
    V = model.fc.out_features
    for ep in range(epochs):
        total = 0.0
        for i in range(0, len(data) - seq_len - 1):
            x = data[i:i+seq_len].unsqueeze(0)
            y = data[i+1:i+seq_len+1].unsqueeze(0)
            logits, _ = model(x)
            loss = crit(logits.reshape(-1, V), y.reshape(-1))
            opt.zero_grad()
            loss.backward()
            opt.step()
            total += loss.item()
        if ep % 10 == 0:
            print(f'epoch {ep} loss {total:.3f}')

print('train_char_rnn 입력 값: seq_len=16, epochs=40, lr=1e-2')
train_char_rnn(char_model, data)


train_char_rnn 입력 값: seq_len=16, epochs=40, lr=1e-2
epoch 0 loss 760.931
epoch 10 loss 551.737
epoch 20 loss 454.281
epoch 30 loss 444.853
epoch 40 loss 399.881
epoch 50 loss 428.366
epoch 60 loss 455.515
epoch 70 loss 416.916
epoch 80 loss 383.602
epoch 90 loss 424.760


In [90]:
# 3. generate_text 함수와 예시
def generate_text(model, start='h', length=80):
    """시작 문자와 길이를 받아 문자 시퀀스를 생성."""
    if start not in stoi:
        raise ValueError('start "{}" is not in the vocabulary'.format(start))
    model.eval()
    x = torch.tensor([[stoi[start]]])
    h = None
    out = [start]
    for _ in range(length):
        with torch.no_grad():
            logits, h = model(x, h)
            probs = torch.softmax(logits[0, -1], dim=0)
            idx = torch.multinomial(probs, 1).item()
        out.append(itos[idx])
        x = torch.tensor([[idx]])
    return ''.join(out)

sample_start = 'h'
sample_length = 80
print(f"generate_text 입력 값: start='{sample_start}', length={sample_length}")
generated = generate_text(char_model, sample_start, sample_length)
print('generate_text 출력 샘플:')
print(generated)


generate_text 입력 값: start='h', length=80
generate_text 출력 샘플:
he modencies bal the modeng nthe modenting works bal the modencies buileveal leve



## 3) RNN 긴 문맥 한계 체험
`[FLAG, noise..., 9, FLAG]` 패턴 데이터로 **기억 유지 능력**을 실험합니다.

- **문제 정의**: 시퀀스 첫 토큰(FLAG)과 마지막 토큰(같은 FLAG) 사이에 긴 잡음 구간과 구분자(DELIM=9)가 끼어 있습니다. 모델이 끝에서 처음 FLAG를 맞혀야 합니다.
- **용어 정리**
  - `FLAG`: 시작/종료에서 짝을 이루는 신호 값(여기서는 1 또는 2).
  - `DELIM`: 잡음 구간과 마지막 FLAG 사이를 구분하는 구분자 토큰(여기서는 9).
  - `noise`: FLAG·DELIM이 아닌 0~8 사이 랜덤 숫자 토큰들이며, 기억을 방해하는 역할을 합니다.
- **왜 어렵나?** 짧은 잡음 구간에서는 FLAG 정보를 유지하기 쉽지만, 구간이 길어질수록 잦은 비관련 토큰을 거쳐야 하므로 은닉 상태가 쉽게 잊혀집니다.
- **실험 절차**
  1. 데이터 생성 (FLAG/잡음/구분자 설명 포함)
  2. 미니배치 구성 및 패딩 방식 확인
  3. 단순 RNN 학습 (짧은 구간만 사용)
  4. 다양한 잡음 길이에 대해 정확도 비교 (짧은 vs 긴)
  5. 대표 시퀀스를 직접 넣어 예측/정답을 살펴보기
- **기대 관찰**: 짧은 구간에서는 높은 정확도를 보이지만, 훈련 때 보지 못한 긴 구간으로 가면 정확도가 눈에 띄게 하락합니다.

In [91]:

# 숫자 토이 데이터 설정
VOCAB = list(range(10))
DELIM = 9
FLAGS = [1, 2]

print('토이 데이터 토큰 집합 (0~9):', VOCAB)
print('FLAG 후보:', FLAGS, '→ 시작/종료 신호 역할')
print('DELIM 토큰:', DELIM, '→ 잡음과 마지막 FLAG 사이의 구분자')
print('noise 토큰: FLAG/DELIM을 제외한 0~8 사이 숫자를 무작위로 사용')

토이 데이터 토큰 집합 (0~9): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
FLAG 후보: [1, 2] → 시작/종료 신호 역할
DELIM 토큰: 9 → 잡음과 마지막 FLAG 사이의 구분자
noise 토큰: FLAG/DELIM을 제외한 0~8 사이 숫자를 무작위로 사용


In [92]:

# 1. make_sample 함수와 예시
def make_sample(min_noise=4, max_noise=12):
    """[FLAG, noise..., 9, FLAG] 시퀀스 하나 생성."""
    flag = random.choice(FLAGS)
    noise = [random.randint(0, 8) for _ in range(random.randint(min_noise, max_noise))]
    # noise는 FLAG/DELIM이 아닌 0~8 범위 숫자로 구성됩니다.
    return [flag] + noise + [DELIM, flag]

sample_seq = make_sample()
print('make_sample 입력 min_noise=4, max_noise=12 (기본값)')
print('make_sample 출력 시퀀스 길이:', len(sample_seq))
print('make_sample 출력 시퀀스:', sample_seq)
print('  구조 = [시작 FLAG, noise(0~8 랜덤)…, DELIM(9), 마지막 FLAG(정답)]')

make_sample 입력 min_noise=4, max_noise=12 (기본값)
make_sample 출력 시퀀스 길이: 14
make_sample 출력 시퀀스: [1, 7, 5, 7, 0, 5, 1, 6, 0, 3, 4, 1, 9, 1]
  구조 = [시작 FLAG, noise(0~8 랜덤)…, DELIM(9), 마지막 FLAG(정답)]


In [93]:

# 2. batchify 함수와 예시
def batchify(batch_size=32, min_noise=4, max_noise=12):
    """가변 길이 샘플들을 0 패딩으로 (x, y, lengths) 배치 구성."""
    batch = [make_sample(min_noise, max_noise) for _ in range(batch_size)]
    maxlen = max(len(s) for s in batch)
    x = torch.full((batch_size, maxlen - 1), 0).long()
    y = torch.full((batch_size, maxlen - 1), 0).long()
    lengths = []
    for i, seq in enumerate(batch):
        inp, tgt = seq[:-1], seq[1:]
        seq_len = len(inp)
        x[i, :seq_len] = torch.tensor(inp)
        y[i, :seq_len] = torch.tensor(tgt)
        lengths.append(seq_len)
    return x, y, torch.tensor(lengths)

bx, by, blen = batchify(batch_size=4, min_noise=3, max_noise=5)
print('batchify 입력: batch_size=4, noise 길이 범위=[3, 5]')
print('batchify 출력 x 텐서 모양:', tuple(bx.shape))
print('batchify 출력 y 텐서 모양:', tuple(by.shape))
print('batchify 출력 각 시퀀스 길이:', blen.tolist())
print('batchify 첫 번째 입력 시퀀스:', bx[0, :blen[0]].tolist())
print('batchify 첫 번째 타깃 시퀀스:', by[0, :blen[0]].tolist())

batchify 입력: batch_size=4, noise 길이 범위=[3, 5]
batchify 출력 x 텐서 모양: (4, 7)
batchify 출력 y 텐서 모양: (4, 7)
batchify 출력 각 시퀀스 길이: [7, 5, 5, 5]
batchify 첫 번째 입력 시퀀스: [1, 6, 6, 5, 6, 4, 9]
batchify 첫 번째 타깃 시퀀스: [6, 6, 5, 6, 4, 9, 1]


In [94]:

# 3. SimpleRNNLM 클래스와 예시
class SimpleRNNLM(nn.Module):
    """단순 RNN 언어모델."""
    def __init__(self, vocab_size=10, hidden=16):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden)
        self.rnn = nn.RNN(hidden, hidden, batch_first=True)
        self.fc = nn.Linear(hidden, vocab_size)

    def forward(self, x, h=None):
        x = self.embed(x)
        out, h = self.rnn(x, h)
        return self.fc(out), h

seq_model = SimpleRNNLM()
batch_logits, _ = seq_model(bx)
print('SimpleRNNLM 입력 배치 모양:', tuple(bx.shape))
print('SimpleRNNLM 출력 로짓 모양:', tuple(batch_logits.shape))
print('SimpleRNNLM 출력 마지막 시퀀스 로짓 상위 5개:', [round(v, 4) for v in batch_logits[0, blen[0]-1, :5].detach().cpu().tolist()])

SimpleRNNLM 입력 배치 모양: (4, 7)
SimpleRNNLM 출력 로짓 모양: (4, 7, 10)
SimpleRNNLM 출력 마지막 시퀀스 로짓 상위 5개: [0.0472, 0.1179, -0.2466, -0.429, -0.0744]


In [95]:

# 4. train_short_context 함수와 예시
def train_short_context(model, epochs=6, steps_per_epoch=120, clip=1.0, noise_range=(4, 12)):
    """noise_range 범위(짧은 구간) 데이터로 학습."""
    torch.manual_seed(0)
    random.seed(0)
    model.train()
    opt = torch.optim.Adam(model.parameters(), lr=0.01)
    crit = nn.CrossEntropyLoss()
    noise_min, noise_max = noise_range
    print(f'train_short_context: noise_len 범위={noise_range}, epochs={epochs}, steps/epoch={steps_per_epoch}')
    for ep in range(epochs):
        total = 0.0
        for _ in range(steps_per_epoch):
            x, y, lengths = batchify(32, noise_min, noise_max)
            logits, _ = model(x)
            last_idx = lengths - 1
            batch_ids = torch.arange(x.size(0))
            final_logits = logits[batch_ids, last_idx]
            final_targets = y[batch_ids, last_idx]
            loss = crit(final_logits, final_targets)
            opt.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            opt.step()
            total += loss.item()
        print(f'  epoch {ep:02d} 평균 loss {total/steps_per_epoch:.4f}')
    print('train_short_context 완료: 모델 가중치가 갱신되었습니다.')

train_short_context(seq_model, epochs=5, steps_per_epoch=120, noise_range=(4, 12))

train_short_context: noise_len 범위=(4, 12), epochs=5, steps/epoch=120
  epoch 00 평균 loss 0.7724
  epoch 01 평균 loss 0.7032
  epoch 02 평균 loss 0.6988
  epoch 03 평균 loss 0.7011
  epoch 04 평균 loss 0.7002
train_short_context 완료: 모델 가중치가 갱신되었습니다.


In [96]:

# 5. eval_on_length 함수와 길이별 비교
def eval_on_length(model, noise_len=80, trials=200):
    """noise_len 길이로 평가: DELIM 직후 FLAG를 맞추는 정확도."""
    model.eval()
    corr = 0
    random.seed(1)
    torch.manual_seed(1)
    with torch.no_grad():
        for _ in range(trials):
            seq = [random.choice(FLAGS)] + [random.randint(0, 8) for _ in range(noise_len)] + [DELIM]
            x = torch.tensor(seq).unsqueeze(0)
            logits, _ = model(x)
            pred = logits[0, -1].argmax().item()
            gold = seq[0]
            corr += int(pred == gold)
    return corr / trials

probe_lengths = [6, 12, 24, 48, 72, 96]
accuracies = [(L, eval_on_length(seq_model, L, trials=400)) for L in probe_lengths]

print('잡음 길이별 정확도 비교 (trials=400):')
for L, acc in accuracies:
    marker = '← 훈련 분포' if L <= 12 else '← 미학습 길이'
    print(f'  noise_len={L:3d}: 정확도 {acc:.3f} {marker}')
short_avg = sum(acc for L, acc in accuracies if L <= 12) / len([L for L in probe_lengths if L <= 12])
long_avg = sum(acc for L, acc in accuracies if L > 48) / len([L for L in probe_lengths if L > 48])
print(f'  단기 평균(<=12): {short_avg:.3f} / 장기 평균(>=48): {long_avg:.3f}')
print('  → 긴 구간으로 갈수록 정확도가 떨어지는지를 확인하세요.')

잡음 길이별 정확도 비교 (trials=400):
  noise_len=  6: 정확도 0.505 ← 훈련 분포
  noise_len= 12: 정확도 0.505 ← 훈련 분포
  noise_len= 24: 정확도 0.500 ← 미학습 길이
  noise_len= 48: 정확도 0.470 ← 미학습 길이
  noise_len= 72: 정확도 0.542 ← 미학습 길이
  noise_len= 96: 정확도 0.537 ← 미학습 길이
  단기 평균(<=12): 0.505 / 장기 평균(>=48): 0.540
  → 긴 구간으로 갈수록 정확도가 떨어지는지를 확인하세요.


In [77]:

# 6. 잡음 길이별 시퀀스 예시 확인
def inspect_sequence(model, noise_len, trials=1):
    for t in range(trials):
        seq = [random.choice(FLAGS)] + [random.randint(0, 8) for _ in range(noise_len)] + [DELIM]
        x = torch.tensor(seq).unsqueeze(0)
        with torch.no_grad():
            logits, _ = model(x)
            probs = torch.softmax(logits[0, -1], dim=0)
            pred = probs.argmax().item()
            confidence = probs[pred].item()
        print(f'  noise_len={noise_len:3d} | 입력 FLAG={seq[0]} | 예측 FLAG={pred} | 확신도={confidence:.3f}')
        print('    시퀀스 앞부분:', seq[:8], '... 끝부분:', seq[-8:])

print('inspect_sequence: 잡음 길이에 따른 개별 예측 비교')
print('단기 잡음 샘플 (noise_len=10)')
inspect_sequence(seq_model, noise_len=10, trials=3)
print('장기 잡음 샘플 (noise_len=80)')
inspect_sequence(seq_model, noise_len=80, trials=3)

inspect_sequence: 잡음 길이에 따른 개별 예측 비교
단기 잡음 샘플 (noise_len=10)
  noise_len= 10 | 입력 FLAG=1 | 예측 FLAG=2 | 확신도=0.589
    시퀀스 앞부분: [1, 5, 7, 8, 1, 3, 8, 4] ... 끝부분: [1, 3, 8, 4, 6, 8, 7, 9]
  noise_len= 10 | 입력 FLAG=1 | 예측 FLAG=2 | 확신도=0.588
    시퀀스 앞부분: [1, 3, 6, 0, 4, 2, 6, 7] ... 끝부분: [4, 2, 6, 7, 7, 8, 3, 9]
  noise_len= 10 | 입력 FLAG=1 | 예측 FLAG=2 | 확신도=0.589
    시퀀스 앞부분: [1, 5, 2, 3, 2, 2, 4, 8] ... 끝부분: [2, 2, 4, 8, 7, 7, 7, 9]
장기 잡음 샘플 (noise_len=80)
  noise_len= 80 | 입력 FLAG=2 | 예측 FLAG=2 | 확신도=0.590
    시퀀스 앞부분: [2, 1, 2, 2, 2, 1, 6, 7] ... 끝부분: [3, 8, 7, 6, 6, 8, 2, 9]
  noise_len= 80 | 입력 FLAG=2 | 예측 FLAG=2 | 확신도=0.587
    시퀀스 앞부분: [2, 5, 6, 7, 3, 0, 8, 2] ... 끝부분: [0, 1, 2, 3, 0, 8, 0, 9]
  noise_len= 80 | 입력 FLAG=2 | 예측 FLAG=2 | 확신도=0.587
    시퀀스 앞부분: [2, 1, 6, 4, 4, 3, 5, 3] ... 끝부분: [3, 1, 5, 5, 5, 7, 0, 9]
